In [1]:
import tensorflow as tf
import numpy as np

In [21]:
tf.reset_default_graph()

In [29]:
batch_size = 40
n_features = 28*28
n_hidden1 = 500
n_hidden2 = 500
n_coding = 20
n_hidden4 = n_hidden2
n_hidden5 = n_hidden1
n_outputs = n_features
learning_rate = 0.01

In [23]:
X = tf.placeholder(shape=[None, n_features], dtype = tf.float32, name = 'X')
y = tf.placeholder(shape=[None, n_features], dtype = tf.float32, name = 'y')

In [24]:
from functools import partial
initializer = tf.contrib.layers.variance_scaling_initializer()
create_dense = partial(tf.layers.dense, activation = tf.nn.elu, kernel_initializer = initializer)

In [25]:
hidden1 = create_dense(inputs = X, units = n_hidden1)
hidden2 = create_dense(inputs = hidden1, units = n_hidden2)
codings_mean = create_dense(inputs = hidden2, units = n_coding, activation = None)
codings_gamma = create_dense(inputs = hidden2, units = n_coding, activation = None)
sampling = tf.random_normal(dtype=tf.float32,shape = tf.shape(codings_gamma))
codings = codings_mean + tf.exp(0.5*codings_gamma)*sampling
hidden4 = create_dense(inputs = codings, units = n_hidden4)
hidden5 = create_dense(inputs = hidden4, units = n_hidden5)
logits = create_dense(inputs = hidden5, activation = None, units = n_outputs)
outputs = tf.nn.sigmoid(logits)

In [26]:
latent_loss = 0.5 * tf.reduce_sum(tf.square(codings_mean) + tf.exp(codings_gamma)- (1 + codings_gamma))

In [27]:
reconstruction_loss = tf.nn.sigmoid_cross_entropy_with_logits(logits = logits , labels=y)

In [28]:
loss = latent_loss + reconstruction_loss

In [30]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

In [31]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

### At Inference time, feed codings = np.random.normal(shape = [n_digits, n_coding])